In [ ]:
#
# Revision: 11/13/2021
# 1. Minor print text updates
# 2. Added new column name verification for fix-up.
# 3. Added an informational message for those columns exist in the output file
#    but not in the input file.
# 4. Comments updated.
#
# Revision: 11/12/2021
# 1. Destructing the input/output file objects after use, so that the files seem
#    to be closed.
# 2. "fixupDict" Updated.
# 3. Added column name verification and allow termination after verification fails.
#
# Revision: 11/7/2021
# 1. Used pandas.ExcelFile, ExcelFile.sheet_names and ExcelFile.parse() to avoid
#    re-reading Excel files.
# 2. Added a simple column count validation after each concatenation to avoid
#    unexpected column additions due to column name typos.
# 3. Minor comment updates
#

import os
import sys
import pandas
pandas.__version__

# Default paths. Feel free to modify
defaultInFolder = "C:/work/InputFolder"
defaultOutFileFullName = "C:/work/Phase 3 buildings.xlsx"

# Some files (both input and output files) may have minor typos in column names.
# This is the fixup dictionary to correct these column names.
fixupDict = {
    "LOCATION / DESCRIPTION"    : "LOCATION DESCRIPTION",
    "CONDITIONRATING"           : "CONDITION RATING",
    "CONDITION\nRATING"         : "CONDITION RATING",
    "PRIORITYRATING"            : "PRIORITY RATING",
    "PRIORITY\nRATING"          : "PRIORITY RATING",
    "Component"                 : "COMPONENT",
    "Units"                     : "UNITS",
    "n longer term."            : "EFFECTIVE AGE",
    "Unit Cost (Remove)"        : "Unit Cost (to be removed)",
    "Unit Cost (remove)"        : "Unit Cost (to be removed)",
    "unit Cost (to be removed)" : "Unit Cost (to be removed)"
}

def AppendNewToFileName(orgFileName):
    fileNameWithoutExt, fileExt = os.path.splitext(orgFileName)
    return fileNameWithoutExt + "-new" + fileExt

def ReplaceNewLineForPrint(orgString):
    return orgString.replace("\n", "\\n")

def Preprocess(Data, OutDataColumns = None):
    # Parameters:
    #   Data: (Required)
    #     The DataFrame object to pre-process. It can be either the DataFrame
    #     of the input file's worksheet or that of the output file's worksheet.
    #
    #   OutDataColumns: (Optional)
    #     A list object that has the output columns. When provided, this function
    #     assumes the Data parameter is the input file's DataFrame object, and
    #     will verify that each regular (non-year) column of the input file (Data)
    #     exists in the output file (OutDataColumns).

    # This function walks through all the year columns to figure out
    # the year range and return this range in the end. Prepare an initial
    # range here to compare during the walk-through.
    inYearMin, inYearMax = 2100, 1900

    # Some Excel files come with redundant columns like "Unnamed: nn".
    # They might be part of Excel.Workbook.Worksheet.UsedRange, but are
    # not actually used. They will be removed by this function. This is
    # the counter of such columns. It's used for trace output at the end
    # of this function.
    UnnamedCols = 0

    # A list to track what columns exist in the output file but not in the input file.
    OutDataColumnsCopy = OutDataColumns

    # Walk through the columns in the reversed order, so that column deletion
    # won't affect the the index-based column walk-through.
    for col in reversed(range(len(Data.columns))):
        colName = Data.columns[col]

        # For regular columns, colName is a "str"-typed object. For year columns
        # like "2021", colName is an "int"-typed object instead of "str"-typed.
        # Calling str functions like startswith() for an "int"-typed object will
        # get exceptions, thus type-check is needed :(

        if isinstance(colName, int):
            # Input files may have more year columns than the output file.
            # Remove these columns so that the result of concat() won't have them.
            # TODO: maybe future years from the input files can be reserved.
            if any(outYearRange):
                if colName < outYearRange[0] or colName > outYearRange[1]:
                    print("Deleting column [%s]" % (colName))
                    Data.drop(columns={colName}, inplace=True)
                    continue

            # Remember the year range. For output file, it will be returned and used during
            # input file processing later. For input file, it's only used for print.
            if colName < inYearMin: inYearMin = colName
            if colName > inYearMax: inYearMax = colName

        elif isinstance(colName, str):
            # Check if the column name needs to be fixed up. (i.a. can be found in the fix-up dict)
            newColName = fixupDict.get(colName)
            if newColName is not None:
                # This column needs to be fixed up. However, if the new column name already exists,
                # DataFrame.rename() won't fail, but give us two columns of the duplicated name.
                # So we need to verify the new column name before calling rename().
                if newColName not in Data.columns:
                    # The new column name doesn't exist yet. So it's safe to go ahead to call rename() now.
                    print("Fixing up column name [%s] => [%s]" \
                        % (ReplaceNewLineForPrint(colName), newColName))
                    Data.rename(columns={colName:newColName}, inplace=True)
                    # Use the new name for the rest processing
                    colName = newColName
                else:
                    # Unfortunately, the new column name does exist. There's not much we can do here.
                    # Just print the warning and leave it. If this is an input file, this column will
                    # be caught as a redundant column later by "if colName not in OutDataColumns"
                    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
                    print("Cannot fix up column name [{0}] => [{1}] because [{1}] exists already." \
                        .format(ReplaceNewLineForPrint(colName), newColName))

            # Some Excel files come with redundant columns like "Unnamed: nn".
            # They might be part of Excel.Workbook.Worksheet.UsedRange, but are
            # not actually used. Remove them to avoid seeing them after concat().
            if colName.startswith("Unnamed:"):
                UnnamedCols += 1
                Data.drop(columns={colName}, inplace=True)
                continue

            # If the OutDataColumns parameter is provided by the caller, the Data parameter
            # should be the input data, so we verify that each regular (non-year) column of
            # the input file exists in the output file (OutDataColumns).
            if OutDataColumns is not None:
                if colName not in OutDataColumns:
                    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
                    print("Input file column [%s] is not found in the output file. 'fixupDict' might need to be updated." \
                        % (ReplaceNewLineForPrint(colName)))
                    answer = input("Do you want to continue? (Answer 'Y' to continue. Otherwise, the program will stop.)")
                    if answer.casefold() != "y":
                        sys.exit(0)

        else:
            # Only int and str types are expected from this DataFrame.columns[col].
            print("Unexpected column name type:", type(colName))

        # Another verification can be done to check if the input file has
        # missed any non-year columns of the output file (except "facility name").
        # This is done by the following steps
        #  1) getting a copy of the output column names first;
        #  2) removing every input column name from the list copy;
        #  3) checking the remaining list after all the input columns are processed.
        # Here is step 2.
        if OutDataColumnsCopy is not None:
            if colName in OutDataColumnsCopy:
                OutDataColumnsCopy.remove(colName)

    # This is just informational. Since "Facility Name" and some difference in the year
    # columns are expected. Also, some columns might be optional too (true?)
    print("Columns that exist in the output file but NOT in this file:", OutDataColumnsCopy)

    if UnnamedCols > 0 : print(f"{UnnamedCols} unnamed columns deleted")
    print(f"Year Range: {inYearMin} ~ {inYearMax}")
    return (inYearMin, inYearMax)


################################
#         Main script
################################

print("Default Input Folder: ", defaultInFolder)
print("Default Output File: ", defaultOutFileFullName)
print("(Note: The actual output file will be %s)" \
    % {AppendNewToFileName(defaultOutFileFullName)})
print()

# User prompts to get the non-default file paths
inFolder = input("Input folder (Press Enter to use the default):")
if inFolder=="" : inFolder = defaultInFolder
outFileFullName = input("Output file (Press Enter to use the default):")
if outFileFullName=="" : outFileFullName = defaultOutFileFullName

# Process the output file
print("\n================================================================")
print("Opening the output file...", outFileFullName)
outFile = pandas.ExcelFile(outFileFullName)
print("Reading the first sheet as the data to join...")
outData = outFile.parse()
print("Read %d records" % (len(outData.index)))

outYearRange = []
outYearRange = Preprocess(outData)
# TODO: what if the output file has no any year columns?

# Try to read the facility name lookup table (with no column header)
# from the second sheet of the output file.
#   Column 1 (no header): FacilityName
#   Column 2 (no header): FileName
if len(outFile.sheet_names) >= 2:
    print("Reading the second sheet as the facility name table...")
    ffDict = outFile.parse(sheet_name=len(outFile.sheet_names)-1,\
                header=None, names=["FacilityName","FileName"], \
                index_col="FileName")
    print("Read %d records" % (len(ffDict.index)))

    # Convert it to a nested dict, which looks as follows
    # {"FacilityName" : {File1:Facility1, File2:Facility2, ...}}
    ffDict = ffDict.to_dict()
else:
    print("The output file doesn't have a facility name lookup table")
    # Build an empty dict as if the second sheet existed but had no content
    ffDict = {"FacilityName" : {}}

# As all the data we need in the output file has been read into memory,
# destroy the ExcelFile object now. This is supposed to close the file handle.
# Note that it's not deleting the file on the disk.
del outFile

# Process each input file in the input folder
for dirpath, dirs, inFiles in os.walk(inFolder):
    for inFileIndex in range(len(inFiles)):
        # Get the full file name
        inFileName = inFiles[inFileIndex]
        inFileFullName = os.path.join(dirpath, inFileName)
        print("\n================================================================")
        print("%d/%d Input File: " % (inFileIndex + 1, len(inFiles)), inFileFullName)

        # Look up for a file name to facility name mapping
        facilityName = ffDict.get("FacilityName").get(inFileName, inFileName)
        print("Facility Name: ", facilityName)

        # Read the input data from the last sheet of the input file
        inFile = pandas.ExcelFile(inFileFullName)
        inData = inFile.parse(sheet_name=len(inFile.sheet_names)-1)
        print("Read %d records from sheet '%s'." \
              % (len(inData.index), inFile.sheet_names[len(inFile.sheet_names)-1]))

        # As all the data we need in the input file has been read into memory,
        # destroy the ExcelFile object now. This is supposed to close the file handle.
        # Note that it's not deleting the file on the disk.
        del inFile

        # Verify the column names (e.g. column name fixups, removing redudant columns, etc.)
        # The output data is provided too to verify that all input columns exist in the output columns.
        print("Preprocessing the input data...")
        Preprocess(inData, outData.columns.to_list())

        # TODO: make sure the "Facility Name" column doesn't exist in the input file already.
        print("Adding the 'Facility Name' column...")
        inData.insert(loc=0, column="Facility Name", value=facilityName)

        oldOutDataColumnCount = len(outData.columns)

        print("Concatenating...")
        outData = pandas.concat([outData, inData])

        # If new columns are added from the input file, give a warning. This is not supposed
        # to happen if the column name verification earlier succeeded. Leaving this warning
        # message here as another level verification.
        if (oldOutDataColumnCount != len(outData.columns)):
            # TODO: find out the actual column addition
            print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print("!!! Column count changed after concatenation. !!!")
            print("Check column name typos, update 'fixupDict' in the script and then start over.")

        print("Done")

# Save as a new file
outFileFullName2 = AppendNewToFileName(outFileFullName)
print("\n================================================================")
print("Saving the final results to ", outFileFullName2)
outData.to_excel(outFileFullName2, index=False)
print("Completed")
